In [1]:
!pip install tensorflow==2.11.0

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install git+https://github.com/VenkateshwaranB/stellargraph.git

  Cloning https://github.com/VenkateshwaranB/stellargraph.git to /tmp/pip-req-build-s4fjwntf
  Running command git clone --filter=blob:none --quiet https://github.com/VenkateshwaranB/stellargraph.git /tmp/pip-req-build-s4fjwntf
  Resolved https://github.com/VenkateshwaranB/stellargraph.git to commit ce29510c83d924561c9e6054641d02a5abdeaaf5
  Preparing metadata (setup.py) ... done
  Created wheel for stellargraph: filename=stellargraph-1.3.0b0-py3-none-any.whl size=431853 sha256=d0682c3c1a22b8f2058bb2548aa0cbc0b326a39098dcdd93070b6b8b0572c704
  Stored in directory: /tmp/pip-ephem-wheel-cache-j4b20c46/wheels/f3/06/0f/089f69af27d308a1830638f855b6c5755311d8ffc451de9980
Successfully built stellargraph


In [4]:
import stellargraph as sg

try:
    sg.utils.validate_notebook_version("1.2.1")
except AttributeError:
    raise ValueError(
        f"This notebook requires StellarGraph version 1.2.1, but a different version {sg.__version__} is installed.  Please see <https://github.com/stellargraph/stellargraph/issues/1172>."
    ) from None

<div class="alert alert-block alert-danger">This notebook is designed for an older StellarGraph version 1.2.1 and may not function correctly with the newer installed version 1.3.0b. Please see: <a href="https://github.com/stellargraph/stellargraph/issues/1172">https://github.com/stellargraph/stellargraph/issues/1172</a>.</div>

<ipython-input-4-9f0eb37542e2>:4: DeprecationWarning: This notebook is designed for an older StellarGraph version 1.2.1 and may not function correctly with the newer installed version 1.3.0b. Please see: <https://github.com/stellargraph/stellargraph/issues/1172>.
  sg.utils.validate_notebook_version("1.2.1")


In [16]:
import matplotlib.pyplot as plt

from sklearn.manifold import TSNE

import os
import networkx as nx
import numpy as np
import pandas as pd
from tensorflow import keras

from stellargraph import StellarGraph
from stellargraph.data import BiasedRandomWalk
from stellargraph.data import UnsupervisedSampler
from stellargraph.data import BiasedRandomWalk
from stellargraph.mapper import Node2VecLinkGenerator, Node2VecNodeGenerator
from stellargraph.layer import Node2Vec, link_classification

from stellargraph import datasets
from IPython.display import display, HTML

%matplotlib inline

In [5]:
from stellargraph.mapper import (
    CorruptedGenerator,
    FullBatchNodeGenerator,
    GraphSAGENodeGenerator,
    HinSAGENodeGenerator,
    ClusterNodeGenerator,
)
from stellargraph import StellarGraph
from stellargraph.layer import GCN, DeepGraphInfomax, GraphSAGE, GAT, APPNP, HinSAGE

from stellargraph import datasets
from stellargraph.utils import plot_history

import pandas as pd
from matplotlib import pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from IPython.display import display, HTML

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras import Model

In [6]:
df_edges = pd.read_csv('/content/drive/MyDrive/graph_ISB_edges.csv', index_col=0)
df_edges

,source,target
0,0,0
1,0,1
2,0,2
3,0,17
4,0,19
...,...,...
153688,1103,1103
153689,1106,1106
153690,1109,1109
153691,1114,1114


In [7]:
node_features_df = pd.read_csv('/content/drive/MyDrive/node_features_hashtag_df.csv')
node_features_df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.189092,-0.059448,0.036934,-0.163505,0.014944,-0.042785,0.065283,0.223801,-0.252894,-0.190002,...,0.072798,0.023225,0.068635,-0.076926,0.118267,-0.121892,-0.053214,-0.071568,0.132806,0.221732
1,-0.189092,-0.059448,0.036934,-0.163505,0.014944,-0.042785,0.065283,0.223801,-0.252894,-0.190002,...,0.072798,0.023225,0.068635,-0.076926,0.118267,-0.121892,-0.053214,-0.071568,0.132806,0.221732
2,-0.189092,-0.059448,0.036934,-0.163505,0.014944,-0.042785,0.065283,0.223801,-0.252894,-0.190002,...,0.072798,0.023225,0.068635,-0.076926,0.118267,-0.121892,-0.053214,-0.071568,0.132806,0.221732
3,-0.189092,-0.059448,0.036934,-0.163505,0.014944,-0.042785,0.065283,0.223801,-0.252894,-0.190002,...,0.072798,0.023225,0.068635,-0.076926,0.118267,-0.121892,-0.053214,-0.071568,0.132806,0.221732
4,-0.189092,-0.059448,0.036934,-0.163505,0.014944,-0.042785,0.065283,0.223801,-0.252894,-0.190002,...,0.072798,0.023225,0.068635,-0.076926,0.118267,-0.121892,-0.053214,-0.071568,0.132806,0.221732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,-0.189092,-0.059448,0.036934,-0.163505,0.014944,-0.042785,0.065283,0.223801,-0.252894,-0.190002,...,0.072798,0.023225,0.068635,-0.076926,0.118267,-0.121892,-0.053214,-0.071568,0.132806,0.221732
1226,-0.189092,-0.059448,0.036934,-0.163505,0.014944,-0.042785,0.065283,0.223801,-0.252894,-0.190002,...,0.072798,0.023225,0.068635,-0.076926,0.118267,-0.121892,-0.053214,-0.071568,0.132806,0.221732
1227,-0.189092,-0.059448,0.036934,-0.163505,0.014944,-0.042785,0.065283,0.223801,-0.252894,-0.190002,...,0.072798,0.023225,0.068635,-0.076926,0.118267,-0.121892,-0.053214,-0.071568,0.132806,0.221732
1228,-0.189092,-0.059448,0.036934,-0.163505,0.014944,-0.042785,0.065283,0.223801,-0.252894,-0.190002,...,0.072798,0.023225,0.068635,-0.076926,0.118267,-0.121892,-0.053214,-0.071568,0.132806,0.221732


In [8]:
G = StellarGraph(node_features_df, df_edges)
print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 1230, Edges: 153693

 Node types:
  default: [1230]
    Features: float32 vector, length 768
    Edge types: default-default->default

 Edge types:
    default-default->default: [153693]
        Weights: all 1 (default)
        Features: none


In [30]:
df_final = pd.read_csv('/content/drive/MyDrive/final_data_ISB.csv')

node_event = {}
for index, row in df_final.iterrows():
    value = row['event']
    key = row['target_node']
    node_event[key] = value

    key = row['response_node']
    node_event[key] = value

In [31]:
import json

with open('/content/drive/MyDrive/reverse_tweet_user_index.json', 'r') as file:
    reverse_tweet_user_index = json.load(file)

print(reverse_tweet_user_index)

{'0': '53692193', '1': '1317129865', '2': '855007514621911040', '3': '878087280321335298', '4': '25073877', '5': '727264708776980480', '6': '3493563087', '7': '292929271', '8': '810619093749559296', '9': '17564591', '10': '26642006', '11': '38029205', '12': '807095', '13': '6473022', '14': '187428929', '15': '17261066', '16': '14247236', '17': '964032914626359296', '18': '149249831', '19': '27032207', '20': '21059255', '21': '246939630', '22': '19568591', '23': '469194846', '24': '21829541', '25': '983766825610694656', '26': '16041234', '27': '701725963', '28': '1917731', '29': '7702542', '30': '28785486', '31': '817382752731463680', '32': '1915033663', '33': '133938408', '34': '78515008', '35': '1589251639', '36': '32871086', '37': '958064770019741696', '38': '797229459803926528', '39': '18510860', '40': '132339474', '41': '24165761', '42': '1462969034', '43': '812135885328236551', '44': '817661098988019712', '45': '166751745', '46': '3260518932', '47': '15907183', '48': '755057897583

In [40]:
graph_node_event = {}
nodes = []
for i in range(1230):
    node = reverse_tweet_user_index[str(i)]
    event = node_event[int(node)]
    nodes.append(int(node))
    graph_node_event[int(node)] = event

In [42]:
series = pd.Series(nodes)
series
subjects = series

In [35]:
G.check_graph_for_ml(features=True)

In [14]:
walk_number = 100
walk_length = 5

In [17]:
walker = BiasedRandomWalk(
    G,
    n=walk_number,
    length=walk_length,
    p=0.5,  # defines probability, 1/p, of returning to source node
    q=2.0,  # defines probability, 1/q, for moving to a node away from the source node
)

In [18]:
unsupervised_samples = UnsupervisedSampler(G, nodes=list(G.nodes()), walker=walker)

In [19]:
batch_size = 50
epochs = 2

In [20]:
generator = Node2VecLinkGenerator(G, batch_size)

In [21]:
emb_size = 128
node2vec = Node2Vec(emb_size, generator=generator)

In [22]:
x_inp, x_out = node2vec.in_out_tensors()

In [23]:
prediction = link_classification(
    output_dim=1, output_act="sigmoid", edge_embedding_method="dot"
)(x_out)

link_classification: using 'dot' method to combine node embeddings into edge embeddings


In [24]:
model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss=keras.losses.binary_crossentropy,
    metrics=[keras.metrics.binary_accuracy],
)

In [25]:
history = model.fit(
    generator.flow(unsupervised_samples),
    epochs=epochs,
    verbose=1,
    use_multiprocessing=False,
    workers=4,
    shuffle=True,
)

Epoch 1/2
12832/12832 [==============================] - 125s 10ms/step - loss: 0.7169 - binary_accuracy: 0.5179
Epoch 2/2
12832/12832 [==============================] - 112s 9ms/step - loss: 0.6987 - binary_accuracy: 0.5590


In [26]:
x_inp_src = x_inp[0]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

In [43]:
node_gen = Node2VecNodeGenerator(G, batch_size).flow(subjects.index)
node_embeddings = embedding_model.predict(node_gen, workers=4, verbose=1)

25/25 [==============================] - 0s 5ms/step


In [44]:
np.save('/content/drive/MyDrive/hashtag_node2vec_embeddings.npy', node_embeddings)